In [1]:
import torch

In [2]:
import torch.nn as nn


class SimpleModel(nn.Module):
    def __init__(self, in_ch, out_ch):
        """Регистрация блоков"""
        super().__init__()
        self.fc1 = nn.Linear(in_ch, 32)  # Полносвязный слой 1
        self.fc2 = nn.Linear(32, out_ch, bias=False)  # Полносвязный слой 2
        self.relu = nn.ReLU()  # Функция активации
        
    def forward(self, x):
        """Прямой проход"""
        h = self.fc1(x)
        h = self.relu(h)
        h = self.fc2(h)
        y = self.relu(h)
        return y

In [3]:
import torch.utils.data as data

In [4]:
class GeneratorDataset(data.Dataset):
    def __init__(self, in_size, out_size, num_samples, func='sin'):
        super().__init__()
        self.num_samples = num_samples
        self.in_size = in_size
        self.out_size = out_size
        self.func = func
        
    def __getitem__(self, index):
        x = torch.rand(self.in_size)
        if self.func == 'sin':
            x = torch.sin(x)
        elif self.func == 'cos':
            x = torch.cos(x)
        y = x[:self.out_size].clone()
        return x, y
    
    def __len__(self):
        return self.num_samples

In [5]:
dataset = GeneratorDataset(64, 10, 128*4)
dataloader = data.DataLoader(dataset, batch_size=16)
for x, y in dataloader:
    break

(x.shape, y.shape)

(torch.Size([16, 64]), torch.Size([16, 10]))

In [6]:
# model = SimpleModel(64, 10)

model1 = nn.Sequential(  # TASK1
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 10, bias=False),
    nn.ReLU()
)

l1_loss = nn.L1Loss()
opt = torch.optim.Adam(model1.parameters(), lr=0.001)

In [7]:
for x, y in dataloader:
    opt.zero_grad()
    
    y_pred = model1(x)
    loss = l1_loss(y, y_pred)
    loss.backward()
    print('Loss', loss.item())
    opt.step()

Loss 0.4300653338432312
Loss 0.38091498613357544
Loss 0.4234698414802551
Loss 0.3976514935493469
Loss 0.3855021893978119
Loss 0.3838973939418793
Loss 0.3696579039096832
Loss 0.3432251214981079
Loss 0.3031436800956726
Loss 0.3417479991912842
Loss 0.3398483991622925
Loss 0.2947714924812317
Loss 0.32756882905960083
Loss 0.27878138422966003
Loss 0.2802906930446625
Loss 0.284973680973053
Loss 0.2696714699268341
Loss 0.26130810379981995
Loss 0.250774085521698
Loss 0.2344980537891388
Loss 0.23825153708457947
Loss 0.22902417182922363
Loss 0.23803266882896423
Loss 0.24089765548706055
Loss 0.25799664855003357
Loss 0.24811188876628876
Loss 0.25032320618629456
Loss 0.22613541781902313
Loss 0.2794349193572998
Loss 0.24040766060352325
Loss 0.25948700308799744
Loss 0.2425203025341034


### Task2

In [15]:
class ThreeLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 4)
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        h = self.fc1(x)
        h = self.relu(h)
        h = self.fc2(h)
        h = self.tanh(h)
        h = self.fc3(h)
        y = self.softmax(h)
        
        return y


In [18]:
dataset = GeneratorDataset(256, 4, 128*10)
dataloader = data.DataLoader(dataset, batch_size=16)

print(x.shape, y.shape)

model2 = ThreeLayerModel()

l1_loss = nn.L1Loss()
opt = torch.optim.Adam(model2.parameters(), lr=0.001)

for x, y in dataloader:
    opt.zero_grad()
    
    y_pred = model2(x)
    loss = l1_loss(y, y_pred)
    loss.backward()
    print('Loss', loss.item())
    opt.step()

torch.Size([16, 256]) torch.Size([16, 4])
Loss 0.2399701625108719
Loss 0.25749272108078003
Loss 0.31736573576927185
Loss 0.2625781297683716
Loss 0.26918846368789673
Loss 0.2722329795360565
Loss 0.2567301094532013
Loss 0.285795122385025
Loss 0.3063666820526123
Loss 0.24298465251922607
Loss 0.26729029417037964
Loss 0.284144788980484
Loss 0.2818048298358917
Loss 0.2665916085243225
Loss 0.2687236964702606
Loss 0.3049793541431427
Loss 0.2924326956272125
Loss 0.2954965829849243
Loss 0.24641719460487366
Loss 0.2870866060256958
Loss 0.25466448068618774
Loss 0.2965160608291626
Loss 0.26691359281539917
Loss 0.2788662016391754
Loss 0.2705926299095154
Loss 0.29452767968177795
Loss 0.24601472914218903
Loss 0.27437254786491394
Loss 0.2976800203323364
Loss 0.26413822174072266
Loss 0.28709274530410767
Loss 0.2871045470237732
Loss 0.25708574056625366
Loss 0.26958829164505005
Loss 0.26479747891426086
Loss 0.2608148157596588
Loss 0.2625938653945923
Loss 0.29312407970428467
Loss 0.27680766582489014
Loss 0

### Task3

In [22]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        h = self.conv1(x)
        h = torch.relu(h)
        h = self.pool1(h)

        h = self.conv2(h)
        h = torch.relu(h)
        y = self.pool2(h)

        return y


model3 = SimpleCNN()

input_tensor = torch.randn(1, 3, 19, 19)  # 19x19x3

output_tensor = model3(input_tensor)

output_tensor.shape

torch.Size([1, 16, 4, 4])

### Task4

In [23]:
output_tensor = output_tensor.reshape(1, 256)
output_tensor.shape

torch.Size([1, 256])

In [24]:
y_pred = model2(output_tensor)
y_pred.shape

torch.Size([1, 4])